# Definição

Este notebook tem como principal função estruturar uma forma de organizar dados obtidos por meio de busca de artigos (revisão). Para automatizar este processo, o buscador Google Scholar foi escolhido. Como não estava disponível uma API para extração de dados, foi desenvolvido um Web Crawler para extrair os dados de artigos encontrados pelo buscador, organizando-os em uma planilha eletrônica para manipulação e consulta posterior.

A lista de queries é a principal variável de controle deste notebook, uma vez que define quais serão os termos pesquisados no buscador. Por definição, dado que o objetivo não é uma revisão sistemática, a requisição utiliza como padrão o algoritmo de ordenação por relevância do Google e acessa até os 70 primeiros resultados por query.

In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import date, datetime

BASE_PATH = os.path.dirname(os.getcwd())

In [2]:
# Gerencia os diretórios e caminho de arquivo excel
today = date.today().isoformat()
today_path = f"{BASE_PATH}/data/google_search/{today}"
pages_path = f"{BASE_PATH}/data/google_search/{today}/pages"
excel_path = f"{BASE_PATH}/data/google_search/{today}/search_results.xlsx"

if not os.path.isdir(today_path):
    os.mkdir(today_path)

if not os.path.isdir(pages_path):
    os.mkdir(pages_path)

In [3]:
queries = [
    'agriculture+and+"linear+programming"',
    '"crops+pattern"+and+"linear+programming"',
    '"crop+rotation"+and+"linear+programming"',
    '"land+allocation"+and+"linear+programming"',
    '"land+allocation"+and+"linear+programming"+and+"crop-livestock"'
]

In [5]:
# Controlam a quantidade de resultados em cada página
# e até onde será extraído
# Fixa o download até a página 6, ordenando por meio
# da relevância do algoritmo do Google
per_page, finish_at = 10, 60

for i in range(0, len(queries)):
    query = queries[i]
    
    for page in range(0, finish_at + per_page, per_page):
        # Exibe log de processamento
        page_number = 1 + int(page / per_page)
        print(f"Processando Query {i + 1}, página {page_number}", end="\r")

        # Estrutura a url do google scholar para download de arquivos
        # Utiliza como ordenador a função de relevância do algoritmo do Google
        # Filtra apenas artigos publicados a partir de 2020
        # Não inclui resultados que são apenas citações, todo retorno é de material original
        url = f"https://scholar.google.com/scholar?lr=&q={query}&as_sdt=0,5&as_ylo=2020&as_vis=1&start={page}"
        req = requests.get(url)

        # Instancia web crawler e acessa listagem de resultados da busca
        soup = BeautifulSoup(req.content, "html.parser")
        search_results = soup.findAll("div", class_="gs_r gs_or gs_scl")

        # Não existem resultados na página, encerra o laço para paginação
        if len(search_results) == 0:
            break

        # Salva o arquivo para leitura posterior
        with open(f"{pages_path}/query_{i + 1}_page_{page_number}.html", "w+") as file:
            file.write(str(soup.prettify()))

In [19]:
summary = []
results = []

for i in range(0, len(queries)):
    start_at = datetime.now()
    query = queries[i]
    query_data = []

    for page in range(0, finish_at + per_page, per_page):
        # Exibe log de processamento
        page_number = 1 + int(page / per_page)
        print(f"Processando Query {i + 1}, página {page_number}", end="\r")

        # Instancia web crawler acessando arquivos locais
        file_path = f"{pages_path}/query_{i + 1}_page_{page_number}.html"
        if not os.path.exists(file_path):
            continue

        with open(file_path, "r") as file:
            soup = BeautifulSoup(file, "html.parser")
            search_results = soup.findAll("div", class_="gs_r gs_or gs_scl")

        # Percorre a lista de resultados, armazenando título do trabalho,
        # link de acesso e autores em lista própria
        for div in search_results:
            title = div.find("h3").find("a")
            author = div.find("div", class_="gs_a").text

            query_data.append({
                "Título": title.text.replace("\n", "").replace("  ", "").strip(),
                "Autor": author.replace("\n", "").replace("  ", "").strip(),
                "Link": title["href"]
            })

    # Adiciona leituras da query na lista de processados
    results.append(query_data)

    # Atualiza log de sumarização de busca
    end_at = datetime.now()
    summary.append({
        "Query": i + 1,
        "Texto": query,
        "Início em": start_at.isoformat(),
        "Término em": end_at.isoformat(),
        "Duração (s)": (end_at - start_at).seconds
    })

with pd.ExcelWriter(excel_path, mode="w") as writer:
    # Salva sumário da busca, com textos de queries
    # Inicia variável para união de resultados
    pd.DataFrame(summary).to_excel(writer, sheet_name="summary", index=False)
    union = []

    # Define uma forma de busca na lista de união
    def search(title: str) -> dict | None:
        element = None
        for i in range(0, len(union)):
            if union[i]["Título"] == title:
                element = union[i]
                break
        return element

    for i in range(0, len(results)):
        # Salva query isolada no arquivo excel
        query_data = results[i]
        pd.DataFrame(query_data).to_excel(writer, sheet_name=f"query_{i + 1}", index=False)

        # Faz o processamento na união
        for record in query_data:
            element = search(record["Título"])
            if element is None:
                record["Query"] = i + 1
                union.append(record)
            else:
                element["Query"] = f'{element["Query"]},{i + 1}'

    # Salva união no arquivo excel
    pd.DataFrame(union).to_excel(writer, sheet_name="union", index=False)
print("Processo de listagem de artigos finalizado")

Processo de listagem de artigos finalizado


In [4]:
# Controlam a quantidade de resultados em cada página
# e até onde será extraído
# Fixa o download até a página 6, ordenando por meio
# da relevância do algoritmo do Google
per_page, finish_at = 10, 60
summary = []
results = []

for i in range(0, len(queries)):
    start_at = datetime.now()
    query = queries[i]
    query_data = []

    for page in range(0, finish_at + per_page, per_page):
        # Exibe log de processamento
        page_number = 1 + int(page / per_page)
        print(f"Processando Query {i + 1}, página {page_number}", end="\r")

        # Estrutura a url do google scholar para download de arquivos
        # Utiliza como ordenador a função de relevância do algoritmo do Google
        # Filtra apenas artigos publicados a partir de 2020
        # Não inclui resultados que são apenas citações, todo retorno é de material original
        url = f"https://scholar.google.com/scholar?lr=&q={query}&as_sdt=0,5&as_ylo=2020&as_vis=1&start={page}"
        req = requests.get(url)

        # Instancia web crawler e acessa listagem de resultados da busca
        soup = BeautifulSoup(req.content, "html.parser")
        search_results = soup.findAll("div", class_="gs_r gs_or gs_scl")

        # Não existem resultados na página, encerra o laço para paginação
        if len(search_results) == 0:
            break

        # Salva o arquivo para leitura posterior
        with open(f"{pages_path}/query_{i + 1}_page_{page_number}.html", "w+") as file:
            file.write(str(soup.prettify()))
        
        # Percorre a lista de resultados, armazenando título do trabalho,
        # link de acesso e autores em lista própria
        for div in search_results:
            title = div.find("h3").find("a")
            author = div.find("div", class_="gs_a").text

            query_data.append({
                "Título": title.text,
                "Autor": author,
                "Link": title["href"]
            })

    # Salva artigos extraídos em arquivo excel
    results.append(query_data)
    '''with pd.ExcelWriter(excel_path, mode="a", if_sheet_exists="replace") as writer:
        pd.DataFrame(results).to_excel(writer, sheet_name=f"query_{i + 1}", index=False)'''

    # Atualiza log de sumarização de busca
    end_at = datetime.now()
    summary.append({
        "Query": i + 1,
        "Texto": query,
        "Início em": start_at.isoformat(),
        "Término em": end_at.isoformat(),
        "Duração (s)": (end_at - start_at).seconds
    })

with pd.ExcelWriter(excel_path, mode="w") as writer:
    # Salva sumário da busca, com textos de queries
    # Inicia variável para união de resultados
    pd.DataFrame(summary).to_excel(writer, sheet_name="summary", index=False)
    union = []

    # Define uma forma de busca na lista de união
    def search(title: str) -> dict | None:
        element = None
        for i in range(0, len(union)):
            if union[i]["Título"] == title:
                element = union[i]
                break
        return element

    for i in range(0, len(results)):
        # Salva query isolada no arquivo excel
        query_data = results[i]
        pd.DataFrame(query_data).to_excel(writer, sheet_name=f"query_{i + 1}", index=False)

        # Faz o processamento na união
        for record in query_data:
            element = search(record["Título"])
            if element is None:
                record["Query"] = i + 1
                union.append(record)
            else:
                element["Query"] = f'{element["Query"]},{i + 1}'

    # Salva união no arquivo excel
    pd.DataFrame(union).to_excel(writer, sheet_name="union", index=False)
print("Processo de listagem de artigos finalizado")

Processo de listagem de artigos finalizado
